In [ ]:
from datasets import Dataset
import nltk
nltk.download("stopwords")
from nltk.corpus import stopwords as nltk_stopwords

In [ ]:
stopwords = set(nltk_stopwords.words("french"))

In [ ]:
def lower_strip_sentence(sentence):
    sent = sentence.lower().strip()
    return sent


def get_words_from_sentence(sentence, strip_characters):
    sent = lower_strip_sentence(sentence)
    words = [word.strip(strip_characters) for word in sent.split(" ")]
    return words

def check_stopwords(
    sentence,
    strip_characters,
    stopwords,
):
    if stopwords:
        words = get_words_from_sentence(sentence, strip_characters)
        stopwords_ratio = len([word for word in words if word in stopwords]) / len(
            words
        )
        return stopwords_ratio
    
def check_special_characters(
    sentence,
    special_characters,
):
    sent = lower_strip_sentence(sentence)
    set_special_characters = {char for char in special_characters}
    special_characters_ratio = len(
        [char for char in sent if char in set_special_characters]
    ) / (len(sent) + 1e-10)
    return special_characters_ratio 

In [ ]:
lang = "fr"
ds = Dataset.load_from_disk(f"../Oscar_filtered/{lang}_debug/")
ds.to_pandas().loc[:, ['cond_check_empty', 'cond_check_special_characters', 'cond_check_stopwords', 'cond_check_badwords', 'cond_check_lang_id']].mean()

In [ ]:
df = ds.to_pandas()

In [ ]:
df.head()

In [ ]:
strip_characters = "' 0123456789¯_%$§½¼¾×|†~\"—±′–'°−{}[]·'?,./<>!@#^&*()+-‑=:;，：,...�`→¶.…‘’”"
special_characters = "' 0123456789¯_%$§½¼¾×|†~\"—±′–'°−{}[]·'?,./<>!@#^&*()+-‑=:;，：,...�`→¶.…‘’”"

In [ ]:
df['stopwords_ratio'] = df['text'].apply(lambda x: check_stopwords(sentence=x, strip_characters=strip_characters, stopwords=stopwords))
df['special_characters_ratio'] = df['text'].apply(lambda x: check_special_characters(x, special_characters))

In [ ]:
ax = df.special_characters_ratio.hist(bins=50, figsize=(12, 12))
ax.set_xlabel("special_characters_ratio")
ax.set_ylabel("Frequency")

In [ ]:
ax = df.stopwords_ratio.hist(bins=50, figsize=(12, 12))
ax.set_xlabel("stopwords_ratio")
ax.set_ylabel("Frequency")

In [ ]:
df

In [ ]:
df.loc[df.special_characters_ratio > 0.35]

In [ ]:
df.loc[df.stopwords_ratio > 0.75].iloc[1].text